In [ ]:
import tensorflow as tf
import segmentation_models as sm
import glob
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import keras 

from keras.utils import normalize
from keras.metrics import MeanIoU

print(keras.__version__)
print(tf.__version__)

#Resizing images, if needed
SIZE_X = 128 
SIZE_Y = 128
n_classes=4 #Number of classes for segmentation

#Capture training image info as a list
train_images = []

for directory_path in glob.glob("../data/patches_128_useful/patch_img/"):
    for img_path in glob.glob(os.path.join(directory_path, "*.jpg")):
        print(img_path, sep = ' ')
        img = cv2.imread(img_path, 1)       
        img = cv2.resize(img, (SIZE_Y, SIZE_X))
        train_images.append(img)
       
#Convert list to array for machine learning processing        
train_images = np.array(train_images)
print(train_images.shape)

#Capture mask/label info as a list
train_masks = [] 

for directory_path in glob.glob("../data/patches_128_useful/patch_mask/"):
    for mask_path in glob.glob(os.path.join(directory_path, "*.tif")):
        print(mask_path, sep = ' ')
        mask = cv2.imread(mask_path, 0)       
        mask = cv2.resize(mask, (SIZE_Y, SIZE_X), interpolation = cv2.INTER_NEAREST)  #Otherwise ground truth changes due to interpolation
        train_masks.append(mask)
        
#Convert list to array for machine learning processing          
train_masks = np.array(train_masks)
print(train_masks.shape)

#Encode labels... but multi dim array so need to flatten, encode and reshape
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
n, h, w = train_masks.shape
train_masks_reshaped = train_masks.reshape(-1,1)
train_masks_reshaped_encoded = labelencoder.fit_transform(train_masks_reshaped)
train_masks_encoded_original_shape = train_masks_reshaped_encoded.reshape(n, h, w)

np.unique(train_masks_encoded_original_shape)

#train_images = np.expand_dims(train_images, axis=3)
#train_images = normalize(train_images, axis=1)

train_masks_input = np.expand_dims(train_masks_encoded_original_shape, axis=3)

#Create a subset of data for quick testing
#Picking 10% for testing and remaining for training
from sklearn.model_selection import train_test_split
X1, X_test, y1, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)
# #Further split training data t a smaller subset for quick testing of models
X_train, X_do_not_use, y_train, y_do_not_use = train_test_split(X1, y1, test_size = 0.3, random_state = 0)


#Full Data
# X_train, X_test, y_train, y_test = train_test_split(train_images, train_masks_input, test_size = 0.10, random_state = 0)
print("Class values in the dataset are ... ", np.unique(y_train))  # 0 is the background/few unlabeled 

from keras.utils import to_categorical
train_masks_cat = to_categorical(y_train, num_classes=n_classes)
y_train_cat = train_masks_cat.reshape((y_train.shape[0], y_train.shape[1], y_train.shape[2], n_classes))


test_masks_cat = to_categorical(y_test, num_classes=n_classes)
y_test_cat = test_masks_cat.reshape((y_test.shape[0], y_test.shape[1], y_test.shape[2], n_classes))

# Add print statements to check tensor shapes
print("Input shapes:")
print("X_train shape:", X_train.shape)
print("y_train_cat shape:", y_train_cat.shape)
print("X_test shape:", X_test.shape)
print("y_test_cat shape:", y_test_cat.shape)


###Model 3
BACKBONE3 = 'vgg19'
preprocess_input3 = sm.get_preprocessing(BACKBONE3)

# preprocess input
X_train3 = preprocess_input3(X_train)
X_test3 = preprocess_input3(X_test)

In [ ]:
from keras.models import load_model
### FOR NOW LET US FOCUS ON A SINGLE MODEL

#Set compile=False as we are not loading it for training, only for prediction.
# model1 = load_model('../SavedModels/res34_backbone_200epochs.hdf5', compile=False)
model3 = load_model('../Segmentation_Models/vgg19/CheckPoints/weights-improvement_vgg19.hdf5', compile=False)

# plot graph
#IOU
y_pred3=model3.predict(X_test3)
y_pred3_argmax=np.argmax(y_pred3, axis=3)


#Using built in keras function
#from keras.metrics import MeanIoU
n_classes = 4
IOU_keras = MeanIoU(num_classes=n_classes)  
IOU_keras.update_state(y_test[:,:,:,0], y_pred3_argmax)
print("Mean IoU =", IOU_keras.result().numpy())

In [ ]:
#To calculate I0U for each class...
values = np.array(IOU_keras.get_weights()).reshape(n_classes, n_classes)
print(values)
class1_IoU = values[0,0]/(values[0,0] + values[0,1] + values[0,2] + values[0,3] + values[1,0]+ values[2,0]+ values[3,0])
class2_IoU = values[1,1]/(values[1,1] + values[1,0] + values[1,2] + values[1,3] + values[0,1]+ values[2,1]+ values[3,1])
class3_IoU = values[2,2]/(values[2,2] + values[2,0] + values[2,1] + values[2,3] + values[0,2]+ values[1,2]+ values[3,2])
class4_IoU = values[3,3]/(values[3,3] + values[3,0] + values[3,1] + values[3,2] + values[0,3]+ values[1,3]+ values[2,3])

print("IoU for class1 is: ", class1_IoU)
print("IoU for class2 is: ", class2_IoU)
print("IoU for class3 is: ", class3_IoU)
print("IoU for class4 is: ", class4_IoU)

#Verify the prediction on first image

In [ ]:
plt.imshow(train_images[0, :,:,0], cmap='gray')

In [ ]:
plt.imshow(train_masks[0], cmap='jet')

In [ ]:
import matplotlib.pyplot as plt

# Create a figure with a larger size
fig = plt.figure(figsize=(10, 10))

# Assuming train_masks contains at least 16 mask images
for i in range(16):
    ax = fig.add_subplot(4, 4, i + 1)  # Create a 4x4 grid of subplots
    ax.imshow(train_masks[i+48], cmap='gray') #16 patches make a single image
    ax.set_title(f"Patch {i + 1}")  # Add a title for each subplot

# Adjust spacing between subplots
plt.subplots_adjust(wspace=0.2, hspace=0.4)
plt.show()  # Display all 16 images

In [ ]:
# Test some random images
for j in range(30):
    import random
    test_img_number = random.randint(0, len(X_test3)-1)
    test_img = X_test3[test_img_number]
    ground_truth=y_test[test_img_number]
    test_img_input=np.expand_dims(test_img, 0)
    
    test_img_input3 = preprocess_input3(test_img_input)
    
    test_pred3 = model3.predict(test_img_input3)
    test_prediction3 = np.argmax(test_pred3, axis=3)[0,:,:]
    
    
    plt.figure(figsize=(12, 8))
    plt.subplot(231)
    plt.title('Testing Image')
    plt.imshow(test_img[:,:,0], cmap='gray')
    plt.subplot(232)
    plt.title('Testing Label')
    plt.imshow(ground_truth[:,:,0], cmap='gray')
    plt.subplot(233)
    plt.title('Prediction on test image')
    plt.imshow(test_prediction3, cmap='gray')
    plt.show()

In [ ]:
"""
https://github.com/Vooban/Smoothly-Blend-Image-Patches
"""

import cv2
import numpy as np

from matplotlib import pyplot as plt
import segmentation_models as sm

from smooth_tiled_predictions import predict_img_with_smooth_windowing

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tifffile as tiff
from keras.models import load_model
# Set your backbone, preprocess_input, patch_size, and n_classes here
BACKBONE = 'vgg19'
preprocess_input = sm.get_preprocessing(BACKBONE)
# size of patches
patch_size = 128

# Number of classes 
n_classes = 4

# Specify the directory containing the images
image_directory = "../LargeImages/Original"
mask_directory = "../LargeImages/Mask"

# Define the result directory
result_directory_mask = f'segmentation_results/{BACKBONE}/mask/'
result_directory_segmented = f'segmentation_results/{BACKBONE}/segmented/'
os.makedirs(result_directory_mask, exist_ok=True)
os.makedirs(result_directory_segmented, exist_ok=True)

# List all image files in the directory
image_files = [file for file in os.listdir(image_directory) if file.endswith(".jpg")]

# Load the model
model = load_model(f'../Segmentation_Models/{BACKBONE}/CheckPoints/weights-improvement_{BACKBONE}.hdf5', compile=False)

# Loop through each image file in the directory
for image_file in image_files:
    # Load the image
    img = cv2.imread(os.path.join(image_directory, image_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_img = preprocess_input(img)

    # Load the corresponding mask
    mask_file = os.path.splitext(image_file)[0] + ".tif"
    original_mask = cv2.imread(os.path.join(mask_directory, mask_file))
    # original_mask = cv2.cvtColor(original_mask, cv2.COLOR_BGR2RGB)
    original_mask = original_mask[:, :, 0]

    # Predict using smooth blending
    predictions_smooth = predict_img_with_smooth_windowing(
        input_img,
        window_size=patch_size,
        subdivisions=2,
        nb_classes=n_classes,
        pred_func=(
            lambda img_batch_subdiv: model.predict((img_batch_subdiv))
        )
    )

    final_prediction = np.argmax(predictions_smooth, axis=2).astype(np.uint8)

    # Save the final prediction and original mask
    result_image_file = f'{result_directory_segmented}{os.path.splitext(image_file)[0]}_segmented_{BACKBONE}.tif'
    result_mask_file = f'{result_directory_mask}{os.path.splitext(image_file)[0]}_mask_{BACKBONE}.tif'
    print(result_image_file)
    print(result_mask_file)
    tiff.imwrite(result_image_file, final_prediction)
    tiff.imwrite(result_mask_file, original_mask)
    # Display the images if needed
    plt.figure(figsize=(12, 12))
    plt.subplot(221)
    plt.title('Testing Image')
    plt.imshow(img)
    plt.subplot(222)
    plt.title('Testing Label')
    plt.imshow(original_mask, cmap = 'gray')
    plt.subplot(223)
    plt.title('Prediction with smooth blending')
    plt.imshow(final_prediction, cmap= 'gray')
    plt.show()


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from keras.models import load_model
import tifffile as tiff
# Set your backbone, preprocess_input, patch_size, and n_classes here
BACKBONE = 'vgg19'
preprocess_input = sm.get_preprocessing(BACKBONE)

# size of patches
patch_size = 128

# Number of classes 
n_classes = 4
# Specify the directory containing the images
image_directory = "../LargeImagesNotTested/"

# Define the result directory
result_directory = f'segmentation_results_not_tested/{BACKBONE}/'
os.makedirs(result_directory, exist_ok=True)

# List all image files in the directory
image_files = [file for file in os.listdir(image_directory) if file.endswith(".jpg")]

# Load the model
model = load_model(f'../Segmentation_Models/{BACKBONE}/CheckPoints/weights-improvement_{BACKBONE}.hdf5', compile=False)

# Loop through each image file in the directory
for image_file in image_files:
    # Load the image
    img = cv2.imread(os.path.join(image_directory, image_file))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    input_img = preprocess_input(img)

    # Predict using smooth blending
    predictions_smooth = predict_img_with_smooth_windowing(
        input_img,
        window_size=patch_size,
        subdivisions=2,
        nb_classes=n_classes,
        pred_func=(
            lambda img_batch_subdiv: model.predict((img_batch_subdiv))
        )
    )

    final_prediction = np.argmax(predictions_smooth, axis=2).astype(np.uint8)
    
    # Save the final prediction
    result_image_file = f'{result_directory}{os.path.splitext(image_file)[0]}_segmented_{BACKBONE}.tif'
    print(result_image_file)
    tiff.imwrite(result_image_file, final_prediction)
    
    # Display the original image and the prediction
    plt.figure(figsize=(20, 10))
    plt.subplot(121)
    plt.title('Testing Image')
    plt.imshow(img, cmap='gray')
    plt.colorbar()
    plt.subplot(122)
    plt.title('Prediction with smooth blending')
    plt.imshow(final_prediction, cmap='gray')
    plt.colorbar()
    plt.show()
